In [1]:
import pandas as pd
import pyfpgrowth
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [2]:
# Load the transaction dataset
food_items = pd.read_csv("1662574418893344.csv")
food_items

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin..."
1,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni..."
2,3,sweet chilli almonds,Snack,veg,"almonds whole, egg white, curry leaves, salt, ..."
3,4,tricolour salad,Healthy Food,veg,"vinegar, honey/sugar, soy sauce, salt, garlic ..."
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest..."
...,...,...,...,...,...
395,396,Kimchi Toast,Korean,veg,"cream cheese, chopped kimchi, scallions,count..."
396,397,"Tacos de Gobernador (Shrimp, Poblano, and Chee...",Mexican,non-veg,"poblano chiles, bacon, shrips, red salsa, garl..."
397,398,Melted Broccoli Pasta With Capers and Anchovies,French,non-veg,"broccoli,Bread Crumbs, anchovy fillets, garli..."
398,399,Lemon-Ginger Cake with Pistachios,Dessert,non-veg,"egg yolks,lemon juice, unsalted butter, all pu..."


In [3]:
food_items.isnull().sum()

Food_ID     0
Name        0
C_Type      0
Veg_Non     0
Describe    0
dtype: int64

In [4]:
food_items['Veg_Non'].nunique()

2

In [5]:
food_items['Veg_Non'].unique()

array(['veg', 'non-veg'], dtype=object)

In [6]:
food_items.describe()

,Food_ID
count,400.000000
mean,200.500000
std,115.614301
min,1.000000
25%,100.750000
50%,200.500000
75%,300.250000
max,400.000000


In [7]:
subset = food_items.sample(400, replace=True)
subset["Describe"] = subset["Describe"].apply(lambda x: x.split(","))  # Convert to list
subset

,Food_ID,Name,C_Type,Veg_Non,Describe
159,160,fish salan,Mexican,non-veg,"[King fish, ripe tomatoes, desi ghee, ginge..."
92,93,buldak (hot and spicy chicken),Japanese,non-veg,"[chicken drum sticks/ chicken breast, soy sau..."
49,50,french pork chop,French,veg,"[pork chop, pink pepper corn, green pepper c..."
101,102,dahi chicken,Indian,non-veg,"[dahi, cumin powder, garlic paste, garam ma..."
231,232,apple and walnut cake,Dessert,veg,"[Apple, Eggs, Walnuts, Sugar, Walnuts, Oi..."
...,...,...,...,...,...
29,30,couscous with ratatouille - tangy tomato sauce,French,veg,"[for the cous cous:, plain couscous, extra v..."
351,352,Lamb Shashlik,Indian,non-veg,"[minced lamb marinated with indian spices, ga..."
95,96,methi chicken masala,Indian,non-veg,"[boneless chicken, lemon juice, red chilli p..."
336,337,Thyme-Roasted Sweet Potatoes,Healthy Food,veg,"[sweet potatoes, thyme leaves, olive oil, r..."


In [8]:
patterns = pyfpgrowth.find_frequent_patterns(subset["Describe"], 7)  # Lower threshold
patterns

{(' Baking Soda',): 7,
 (' Baking Soda', ' Honey'): 10,
 (' spring onion',): 7,
 (' fresh cream',): 7,
 (' fresh cream', ' salt'): 8,
 (' tomato puree',): 7,
 (' salt', ' tomato puree'): 7,
 (' clove',): 7,
 (' clove', ' salt'): 9,
 (' white chocolate',): 7,
 (' white wine',): 7,
 (' Ginger',): 7,
 (' Ginger', ' Salt'): 7,
 (' cardamom',): 7,
 (' cardamom', ' salt'): 11,
 (' green coriander',): 7,
 (' green coriander', ' salt'): 8,
 (' salt', ' sesame oil'): 7,
 (' salt', ' sesame oil', ' sugar'): 7,
 (' sesame oil', ' sugar'): 9,
 (' Onion',): 8,
 (' Onion', ' Salt'): 7,
 (' cumin seeds',): 8,
 (' cumin seeds', ' salt'): 9,
 (' bay leaf', ' olive oil'): 8,
 (' bay leaf', ' olive oil', ' salt'): 20,
 (' bay leaf', ' salt'): 15,
 (' bay leaf', ' salt', ' salt'): 10,
 (' vanilla extract',): 8,
 (' brown sugar',): 8,
 (' celery',): 8,
 (' celery', ' salt'): 11,
 (' mozzarella cheese',): 8,
 (' green onions',): 8,
 (' green onions', ' salt'): 7,
 (' green cardamom',): 8,
 (' garlic paste',

In [9]:
rules = pyfpgrowth.generate_association_rules(patterns,1)
rules

{(' Baking Soda',): ((' Honey',), 1.4285714285714286),
 (' fresh cream',): ((' salt',), 1.1428571428571428),
 (' tomato puree',): ((' salt',), 1.0),
 (' clove',): ((' salt',), 1.2857142857142858),
 (' Ginger',): ((' Salt',), 1.0),
 (' cardamom',): ((' salt',), 1.5714285714285714),
 (' green coriander',): ((' salt',), 1.1428571428571428),
 (' salt', ' sesame oil'): ((' sugar',), 1.0),
 (' cumin seeds',): ((' salt',), 1.125),
 (' bay leaf', ' olive oil'): ((' salt',), 2.5),
 (' bay leaf', ' salt'): ((' olive oil',), 1.3333333333333333),
 (' celery',): ((' salt',), 1.375),
 (' garlic paste', ' yellow chilli powder'): ((' olive oil', ' salt'), 1.0),
 (' salt', ' yellow chilli powder'): ((' olive oil',), 1.5),
 (' olive oil', ' yellow chilli powder'): ((' garlic paste', ' salt'), 1.0),
 (' garlic paste', ' olive oil', ' yellow chilli powder'): ((' salt',), 1.0),
 (' onion',
  'chicken'): ((' coriander powder',
   ' cumin powder',
   ' red chilli powder',
   ' salt'), 1.1428571428571428),
 (

In [10]:
# Function to display all rules
def display_rules(rules):
    for rule in rules:
        antecedent = rule
        consequent = rules[rule][0]
        confidence = rules[rule][1]
        print(f"Rule: {antecedent} -> {consequent} (Confidence: {confidence:.2f})")

# Function to filter and display rules related to a specific item
def display_rules_for_itemset(rules, item_name):
    print(f"\nAssociation rules related to '{item_name}':")
    found = False
    item_name = item_name.strip().lower() 
    for rule in rules:
        antecedent = tuple(x.strip().lower() for x in rule)
        consequent = rules[rule][0]
        consequent = consequent.strip().lower() if isinstance(consequent, str) else tuple(x.strip().lower() for x in consequent)  # Clean up consequent items
        confidence = rules[rule][1]
        
        if item_name in antecedent or item_name in consequent:
            found = True
            print(f"Rule: {antecedent} -> {consequent} (Confidence: {confidence:.2f})")
    
    if not found:
        print(f"No rules found for '{item_name}'.")


# Display all rules
print("All Association Rules:")
display_rules(rules)

All Association Rules:
Rule: (' Baking Soda',) -> (' Honey',) (Confidence: 1.43)
Rule: (' fresh cream',) -> (' salt',) (Confidence: 1.14)
Rule: (' tomato puree',) -> (' salt',) (Confidence: 1.00)
Rule: (' clove',) -> (' salt',) (Confidence: 1.29)
Rule: (' Ginger',) -> (' Salt',) (Confidence: 1.00)
Rule: (' cardamom',) -> (' salt',) (Confidence: 1.57)
Rule: (' green coriander',) -> (' salt',) (Confidence: 1.14)
Rule: (' salt', ' sesame oil') -> (' sugar',) (Confidence: 1.00)
Rule: (' cumin seeds',) -> (' salt',) (Confidence: 1.12)
Rule: (' bay leaf', ' olive oil') -> (' salt',) (Confidence: 2.50)
Rule: (' bay leaf', ' salt') -> (' olive oil',) (Confidence: 1.33)
Rule: (' celery',) -> (' salt',) (Confidence: 1.38)
Rule: (' garlic paste', ' yellow chilli powder') -> (' olive oil', ' salt') (Confidence: 1.00)
Rule: (' salt', ' yellow chilli powder') -> (' olive oil',) (Confidence: 1.50)
Rule: (' olive oil', ' yellow chilli powder') -> (' garlic paste', ' salt') (Confidence: 1.00)
Rule: (' 

In [11]:
display_rules_for_itemset(rules, 'red chilli powder')


Association rules related to 'red chilli powder':
Rule: ('onion', 'chicken') -> ('coriander powder', 'cumin powder', 'red chilli powder', 'salt') (Confidence: 1.14)
Rule: ('salt', 'chicken') -> ('coriander powder', 'cumin powder', 'red chilli powder') (Confidence: 1.45)
Rule: ('red chilli powder', 'chicken') -> ('coriander powder', 'cumin powder', 'salt') (Confidence: 2.00)
Rule: ('coriander powder', 'chicken') -> ('cumin powder', 'red chilli powder', 'salt') (Confidence: 2.00)
Rule: ('cumin powder', 'chicken') -> ('coriander powder', 'red chilli powder', 'salt') (Confidence: 2.00)
Rule: ('onion', 'red chilli powder', 'chicken') -> ('coriander powder', 'cumin powder', 'salt') (Confidence: 1.00)
Rule: ('coriander powder', 'onion', 'chicken') -> ('cumin powder', 'red chilli powder', 'salt') (Confidence: 1.00)
Rule: ('cumin powder', 'onion', 'chicken') -> ('coriander powder', 'red chilli powder', 'salt') (Confidence: 1.00)
Rule: ('coriander powder', 'onion', 'red chilli powder', 'chicken

In [12]:
food_items["Describe"]

0      white balsamic vinegar, lemon juice, lemon rin...
1      olive oil, chicken mince, garlic (minced), oni...
2      almonds whole, egg white, curry leaves, salt, ...
3      vinegar, honey/sugar, soy sauce, salt, garlic ...
4      christmas dry fruits (pre-soaked), orange zest...
                             ...                        
395     cream cheese, chopped kimchi, scallions,count...
396    poblano chiles, bacon, shrips, red salsa, garl...
397    broccoli,Bread Crumbs,  anchovy fillets, garli...
398    egg yolks,lemon juice, unsalted butter, all pu...
399    kosher salt, rosemary, garlic, potato, olive o...
Name: Describe, Length: 400, dtype: object

In [17]:
# def get_top_items(food_items, patterns, top_n=50):
    all_items = [item.strip() for items in data["Describe"] for item in items.split(",")]
    unique_items = set([item for item in all_items if item])
    items_in_patterns = set()
    for pattern in patterns:
        items_in_patterns.update(pattern)
        
    return items_in_patterns

96